<a href="https://colab.research.google.com/github/charlesellis1/trump/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
trump_tweets = pd.read_csv('/content/tweets_11-06-2020.csv')

In [ ]:
len(trump_tweets)

55090

In [ ]:
nonrt_tweets = trump_tweets[trump_tweets['isRetweet'] == 'f']
nonrt_tweets.head(3)

,id,text,isRetweet,isDeleted,device,favorites,retweets,date
0,98454970654916608,Republicans and Democrats have both created ou...,f,f,TweetDeck,49,255,2011-08-02 18:07:48
1,1234653427789070336,I was thrilled to be back in the Great city of...,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major...,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58


In [ ]:
nonrt_tweets['year'] = pd.DatetimeIndex(nonrt_tweets['date']).year
nonrt_tweets['month'] = pd.DatetimeIndex(nonrt_tweets['date']).month

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
nonrt2020_tweets = nonrt_tweets[(nonrt_tweets['year'] == 2016) & (nonrt_tweets['month'] == 10)]
nonrt2020_tweets['text_non_url'] = nonrt2020_tweets['text'].str.replace(r"http\S+", "", regex=True)
nonrt2020_tweets['text_new'] = nonrt2020_tweets['text_non_url'].replace(r'^\s*$', np.nan, regex=True)
nonrt2020_tweets = nonrt2020_tweets.drop(columns={'text', 'text_non_url'}).dropna()
nonrt2020_tweets.head(3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,isRetweet,isDeleted,device,favorites,retweets,date,year,month,text_new
1120,792176315478609920,f,f,Twitter for Android,19758,5237,2016-10-29 01:28:46,2016,10,Just landed in Iowa - speaking soon!
1121,788765196584038400,f,f,Twitter for iPhone,14671,7728,2016-10-19 15:34:12,2016,10,It is time to #DrainTheSwamp!
1122,786589172911964161,f,f,Twitter for iPhone,25660,11527,2016-10-13 15:27:27,2016,10,Thank you! #MAGA #AmericaFirst


In [ ]:
#average number of chars in tweets
nonrt2020_tweets['text_len'] = nonrt2020_tweets['text_new'].str.len()
nonrt2020_tweets['text_len'].mean()

96.87525562372188

In [ ]:
#word frequency
word_freq = nonrt2020_tweets.text_new.str.split(expand=True).stack().value_counts()

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(word_freq)

the                                        191
to                                         158
in                                         136
and                                        121
is                                          97
of                                          96
I                                           81
a                                           79
for                                         71
you                                         65
will                                        65
-                                           62
on                                          59
Hillary                                     54
&amp,                                       50
#DrainTheSwamp                              47
Thank                                       45
at                                          44
be                                          39
Clinton                                     39
#BigLeagueTruth                             38
are          

In [ ]:
lst = list(nonrt2020_tweets['text_new'])

In [ ]:
# number of tweets
len(lst)

489

In [ ]:
#number of chars
txt_file = ""
for tweet in lst:
  txt_file += tweet + " "
len(txt_file)

47861

In [ ]:
with open('tweets.txt','w') as f:
  f.write(txt_file)

In [ ]:
#load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [ ]:
#load text
raw_text = load_doc('tweets.txt')
print(raw_text)

# clean
tokens = raw_text.split()
raw_text = ' '.join(tokens)

# organize into sequences of characters
length = 10
sequences = list()
for i in range(length, len(raw_text)):
    # select sequence of tokens
    seq = raw_text[i-length:i+1]
    # store
    sequences.append(seq)
print('Total Sequences: %d' % len(sequences))

Just landed in Iowa - speaking soon! It is time to #DrainTheSwamp! Thank you! #MAGA #AmericaFirst  Yet more evidence of a media-rigged election:  .@mike_pence is doing a great job - so far, no contest! Wow! I hear you Warren, Michigan. Streaming live - join us America. It is time to DRAIN THE SWAMP!Watch:  #ObamacareFail #HillarycareFail  Thank you Grand Rapids, Michigan! #ICYMI- watch:   $25 Million+ raised online in just one week! RECORD WEEK. #DrainTheSwamp Today we set a bigger record. Contribute &gt, Legendary basketball coach Bobby Knight who has 900+ wins, many championships and a gold medal will be introducing...  'Hillary's Two Official Favors To Morocco Resulted In $28 Million For Clinton Foundation' #DrainTheSwamp Join me tomorrow in Michigan!Grand Rapids at 12pm: at 3pm:  Beautiful rally in Albuquerque, New Mexico this evening - thank you. Get out &amp, VOTE! #DrainTheSwampWatch rally:...  """@slh: I follow Mr.Trump at all of his rallies by watching them on  He is a lion-he

In [ ]:
# save sequences to file
out_filename = 'char_sequences.txt'
save_doc(sequences, out_filename)

In [ ]:
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [ ]:
# load

in_filename = 'char_sequences.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')

In [ ]:
# integer encode sequences of characters
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))
sequences = list()
for line in lines:
    # integer encode line
    encoded_seq = [mapping[char] for char in line]
    # store
    sequences.append(encoded_seq)

# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)

Vocabulary Size: 101


In [ ]:
from keras.layers.normalization import BatchNormalization

model = Sequential()
model.add(LSTM(vocab_size, input_shape=(X.shape[1], X.shape[2])))
#model.add(BatchNormalization())
model.add(Dense(vocab_size, activation='softmax'))
#model.add(Dense(vocab_size, activation='selu'))
model.add(Dropout(.2))
# model.add(Dense(vocab_size, activation='selu'))
# model.add(Dropout(.2))
# model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
history=model.fit(X, y, epochs=50, validation_split=0.2, batch_size=132, shuffle=True)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 101)               82012     
_________________________________________________________________
dense_1 (Dense)              (None, 101)               10302     
_________________________________________________________________
dropout_1 (Dropout)          (None, 101)               0         
Total params: 92,314
Trainable params: 92,314
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
1189/1189 [==============================] - 14s 11ms/step - loss: 5.6747 - accuracy: 0.1758 - val_loss: 2.9386 - val_accuracy: 0.2524
Epoch 2/50
1189/1189 [==============================] - 13s 11ms/step - loss: 5.2255 - accuracy: 0.2671 - val_loss: 2.7041 - val_accuracy: 0.2918
Epoch 3/50
1189/1189 [==============================] - 13s 11ms/step - loss: 5.023

In [ ]:
# save the model to file
model.save('model.h5')
# save the mapping
dump(mapping, open('mapping.pkl', 'wb'))

In [ ]:
from pickle import load
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text

# load the model
model = load_model('model.h5')
# load the mapping
mapping = load(open('mapping.pkl', 'rb'))

In [ ]:
# test start of tweet
print(generate_seq(model, mapping, 10, 'COVID', 80))
# test something random
print(generate_seq(model, mapping, 10, 'Joe', 200))
# idk
print(generate_seq(model, mapping, 10, 'Election', 80))

COVID #Rrooked &amp, the Gave out or Crintina Amp. We are going to cover we are going
Joe the mastice in Funtrically in the wint, will be in Cleveland, Ohio- join to #DrainTheSwamp 'Clinton Campaign Trea know HeCriala: what whe will me havered in What THAC YOUR LALIION LASTICNE REALLIOSLI
Elections In be fiver. We wall creese the triem the Turs Tolerrate Thin seding and the p


In [ ]:
print(generate_seq(model, mapping, 10, 'I', 150))
# test something random
print(generate_seq(model, mapping, 10, '#MAGA', 120))
# idk
print(generate_seq(model, mapping, 10, 'Biden', 150))

IS that the for the President to the President to the President to the President to the President to the President to the President to the President to
#MAGA than the President to the President to the President to the President to the President to the President to the Presiden
Biden and the President to the President to the President to the President to the President to the President to the President to the President to the Presi


In [ ]:
print(generate_seq(model, mapping, 10, 'Hillary', 150))

Hillary has done in the final diblial yne thereswen will eve last what whe will me havered in What THAC YOUR LALIION LASTICNE REALLIOSLISTORFLESEtE #DrainThe


In [ ]:
print(generate_seq(model, mapping, 10, '#MAGA', 150))

#MAGA than the President to the President to the President to the President to the President to the President to the President to the President to the Pres


In [ ]:
print(generate_seq(model, mapping, 10, 'I think', 140))

I think for the mayes a be is a disaster the thit tomorrow at 3pm: Thank you Gettors in Clinton on the firusing the becutive and a colation how #Dr
